In [2]:
from marley.utils.ipstartup import *
from marley import tif, config as c
import rasterio
from rasterstats import zonal_stats
import geopandas as gpd
import plotly.express as px
os.chdir(c.localpath)

# align rasters

In [ ]:
# grid3
dstfolder = "grid3"
tgt = "grid3/GRID3_NGA_PopEst_v1_1_mean_float.tif"
tif.align("dmsp/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS_median_stable_lights.tif", tgt, dstfolder)
tif.align("viirs/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG_median_avg_rad.tif", tgt, dstfolder)

In [ ]:
# wpop
dstfolder = "wpop"
tgt = "wpop/WorldPop_GP_100m_pop_median_population.tif"
tif.align("GRID3_NGA_PopEst_v1_1_mean_float.tif", tgt, dstfolder, True)

In [ ]:
# VIIRS
dstfolder = "viirs"
tgt = "viirs/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG_median_avg_rad.tif"
tif.align("grid3/GRID3_NGA_PopEst_v1_1_mean_float.tif", tgt, dstfolder, True)
tif.align("wpop/WorldPop_GP_100m_pop_median_population.tif", tgt, dstfolder, True)
tif.align("dmsp/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS_median_stable_lights.tif", tgt, dstfolder)

In [ ]:
# DMSP
dstfolder = "dmsp"
tgt = "dmsp/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS_median_stable_lights.tif"

tif.align("grid3/GRID3_NGA_PopEst_v1_1_mean_float.tif", tgt, dstfolder, True)
tif.align("wpop/WorldPop_GP_100m_pop_median_population.tif", tgt, dstfolder, True)
tif.align("viirs/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG_median_avg_rad.tif", tgt, dstfolder)

# align vector and raster

In [3]:
# read vector
areas = gpd.read_file("vectors/gadm36_NGA.gpkg")
areas.head(2)

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:33)


,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,NGA,Nigeria,NGA.1_1,Abia,None,NGA.1.1_1,Aba North,None,None,Local Authority,Local Authority,None,None,"MULTIPOLYGON (((7.37961 5.05966, 7.37811 5.056..."
1,NGA,Nigeria,NGA.1_1,Abia,None,NGA.1.2_1,Aba South,None,None,Local Authority,Local Authority,None,None,"MULTIPOLYGON (((7.37811 5.05695, 7.37760 5.056..."


time: 683 ms


In [8]:
# read raster
popfile = "wpop/WorldPop_GP_100m_pop_median_population.tif"
population = rasterio.open(popfile).read()
population[population>0].sum(), population.shape

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:46)


(209758900.0, (1, 11539, 14403))

time: 3.53 s


In [9]:
# get raster data using vector areas
stats = zonal_stats("vectors/gadm36_NGA.gpkg", popfile)
len(stats), stats[0]

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:46)


(775,
 {'min': 10.199530601501465,
  'max': 191.3248291015625,
  'mean': 72.392599384761,
  'count': 2113})

time: 19.3 s


In [10]:
# reformat areas and combine with stats
df = areas.rename(columns=dict(GID_2="code", NAME_2="area"))[["area", "code"]]
df = pd.concat([df, pd.DataFrame.from_records(stats)], axis=1)
df["population"] = df["mean"]*df["count"]
df.population= df.population.astype(int)
d(len(df))
df.head()

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:46)


775

,area,code,count,max,mean,min,population
0,Aba North,NGA.1.1_1,2113,191.324829,72.392599,10.199531,152965
1,Aba South,NGA.1.2_1,5978,294.142273,102.275510,12.347239,611403
2,Arochukw,NGA.1.3_1,45168,42.837994,5.363873,0.929764,242275
3,Bende,NGA.1.4_1,70684,30.097012,3.899136,1.070607,275606
4,Ikwuano,NGA.1.5_1,27836,45.921654,7.086864,2.090636,197269


time: 22 ms


# compare raster and census population

In [11]:
# format census
census = pd.read_excel(r"tables/nga_admpop_2016.xlsx", sheet_name="NGA Admin2 2016 Pop")
census = census.rename(columns={"admin2RefName": "area", "Population2016":"population", "admin2Pcode":"code"})
census["code2"] = census.code.apply(lambda x: f"NGA.{int(x[2:5])}.{int(x[5:8])}_1")
census = census[["code", "code2", "area", "population"]]
d(len(census))
census.head()

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:46)


774

,code,code2,area,population
0,NG001001,NGA.1.1_1,Aba North,138237
1,NG001002,NGA.1.2_1,Aba South,545102
2,NG008001,NGA.8.1_1,Abadam,136810
3,NG015001,NGA.15.1_1,Abaji,124635
4,NG003001,NGA.3.1_1,Abak,189947


time: 154 ms


In [12]:
# THE AREAS DONT MATCH!!!!!!
both = df.merge(census, left_on="code", right_on="code2", how="outer",
                indicator=True, suffixes=["_pop", "_cen"])\
                .sort_values("area_pop")
both.to_excel("temp.xlsx")
both[both._merge!="both"]

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:46)


,area_pop,code_pop,count,max,mean,min,population_pop,code_cen,code2,area_cen,population_cen,_merge
172,Bakassi,NGA.9.4_1,9112.0,24.113644,4.512296,1.635580,41116.0,NaN,NaN,NaN,NaN,left_only
168,Shani,NGA.8.28_1,141443.0,24.187889,1.099146,0.280835,155466.0,NaN,NaN,NaN,NaN,left_only
775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NG009019,NGA.9.19_1,Yala,275903.0,right_only


time: 948 ms


In [13]:
# check %diff after adjusting to same scale as different years
both["cen2"] = both.population_cen*both.population_pop.sum()/both.population_cen.sum()
both.cen2 = both.cen2.fillna(0).astype(int)
both["diffpc"] = (both.cen2/both.population_pop-1)*100
both.drop(columns=["count", "max", "mean", "min"])

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:46)


,area_pop,code_pop,population_pop,code_cen,code2,area_cen,population_cen,_merge,cen2,diffpc
0,Aba North,NGA.1.1_1,152965.0,NG001001,NGA.1.1_1,Aba North,138237.0,both,154230,0.826987
1,Aba South,NGA.1.2_1,611403.0,NG001002,NGA.1.2_1,Aba South,545102.0,both,608168,-0.529111
141,Abadam,NGA.8.1_1,114090.0,NG008001,NGA.8.1_1,Abadam,136810.0,both,152638,33.787361
276,Abaji,NGA.15.1_1,185801.0,NG015001,NGA.15.1_1,Abaji,124635.0,both,139054,-25.159714
38,Abak,NGA.3.1_1,211179.0,NG003001,NGA.3.1_1,Abak,189947.0,both,211923,0.352308
212,Abakalik,NGA.11.1_1,214146.0,NG011001,NGA.11.1_1,Abakaliki,196830.0,both,219602,2.547794
564,Abeokuta South,NGA.28.1_1,377645.0,NG028001,NGA.28.1_1,Abeokuta North,272627.0,both,304168,-19.456633
565,AbeokutaNorth,NGA.28.2_1,299938.0,NG028002,NGA.28.2_1,Abeokuta South,338910.0,both,378120,26.066054
169,Abi,NGA.9.1_1,206710.0,NG009001,NGA.9.1_1,Abi,189484.0,both,211406,2.271782
293,Aboh-Mba,NGA.17.1_1,289575.0,NG017001,NGA.17.1_1,Aboh-Mbaise,262697.0,both,293090,1.213848


time: 40 ms
